In [1]:
import numpy as np
import pandas as pd
from scipy.io import arff
import pandas as pd
import pytempo
import sys

In [2]:
train = arff.loadarff('BirdChicken_TRAIN.arff')
train = pd.DataFrame(train[0])
test = arff.loadarff('BirdChicken_TEST.arff')
test = pd.DataFrame(test[0])
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Columns: 513 entries, att1 to target
dtypes: float64(512), object(1)
memory usage: 80.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Columns: 513 entries, att1 to target
dtypes: float64(512), object(1)
memory usage: 80.3+ KB
None


In [3]:
np_train=train.iloc[:,:-1].to_numpy()
np_train

array([[1.7613808 , 1.7294248 , 1.6788066 , ..., 1.7279482 , 1.7314824 ,
        1.7567164 ],
       [0.39157309, 0.36945283, 0.35483571, ..., 0.25177587, 0.31538894,
        0.35023064],
       [0.52484108, 0.50080229, 0.46619123, ..., 0.38381657, 0.44763553,
        0.48581159],
       ...,
       [0.18251809, 0.19113623, 0.20164256, ..., 0.00297502, 0.08939025,
        0.09625011],
       [1.7840405 , 1.7283977 , 1.6734765 , ..., 1.8390928 , 1.7814412 ,
        1.7244537 ],
       [0.89794077, 0.88142134, 0.86642527, ..., 0.65809375, 0.73796358,
        0.81791532]])

In [4]:
np_train_label=train.iloc[:,-1].to_numpy()
np_train_label=np_train_label.astype(int)
np_train_label

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [5]:
np_test=test.iloc[:,:-1].to_numpy()
np_test

array([[1.4551033 , 1.419576  , 1.3845511 , ..., 1.5018484 , 1.5053846 ,
        1.468697  ],
       [1.8012971 , 1.7569112 , 1.7129602 , ..., 1.8242671 , 1.8236499 ,
        1.8234453 ],
       [1.8400906 , 1.8569621 , 1.8745982 , ..., 1.7356646 , 1.7724973 ,
        1.8133998 ],
       ...,
       [0.05787606, 0.04393106, 0.02600393, ..., 0.07278278, 0.05398408,
        0.03672941],
       [1.5259421 , 1.4698897 , 1.4150402 , ..., 1.5676443 , 1.5527827 ,
        1.5388809 ],
       [1.0052491 , 0.98260008, 0.92280205, ..., 0.98456431, 0.98805458,
        0.96860373]])

In [6]:
np_test_label=test.iloc[:,-1].to_numpy()
np_test_label=np_test_label.astype(int)
np_test_label

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
# Euclidean
def Euclidean():
    min_ed =sys.maxsize
    min_y = sys.maxsize
    result_ed=[]
    ed=[]
    for i in range(len(np_test)):
        q = np_test[i]
        min_ed =sys.maxsize
        min_y = sys.maxsize
        for j in range(len(np_train)):
            c= np_train[j]
            v = np.linalg.norm(q - c)
            if(v<min_ed):
                min_ed = v
                min_y = np_train_label[j]
        result_ed.append(min_y)
    return result_ed

# DTW
def dtw():
    min_dtw =sys.maxsize
    min_y = sys.maxsize
    ud = pytempo.univariate.distances
    result=[]
    dtw=[]
    for i in range(len(np_test)):
        q = np_test[i]
        min_dtw =sys.maxsize
        min_y = sys.maxsize
        for j in range(len(np_train)):
            c= np_train[j]
            v = ud.dtw(q, c)
            if(v<min_dtw):
                min_dtw = v
                min_y = np_train_label[j]
        result.append(min_y)
        dtw.append(min_dtw)  
    return result
def cdtw(w):
    min_cdtw =sys.maxsize
    min_y = sys.maxsize
    result_cdtw=[]
    cdtw=[]
    L = 512
    for i in range(len(np_test)):
        q = np_test[i]
        min_cdtw =sys.maxsize
        min_y = sys.maxsize
        for j in range(len(np_train)):
            c= np_train[j]
            v = ud.cdtw(q, c, w*(L-1))
            if(v<min_cdtw):
                min_cdtw = v
                min_y = np_train_label[j]
        result_cdtw.append(min_y)
        cdtw.append(min_cdtw) 
    return result_cdtw

In [8]:
q1=0
q2=0
q3=0
ud = pytempo.univariate.distances
r1 = Euclidean()
r2 = dtw()
r3 = cdtw(6)
for i in range(len(np_test_label)):
    if r1[i]==np_test_label[i]:
        q1=q1+1
    if r2[i]==np_test_label[i]:
        q2=q2+1
    if r3[i]==np_test_label[i]:
        q3=q3+1   
a1 = q1/len(np_test_label)
a2 = q2/len(np_test_label)
a3 = q3/len(np_test_label)
print(f'Ed: {format(1-a1, ".4f")}')
print(f'DTW(Learned w): {format(1-a3, ".4f")}')
print(f'DTW(w=100): {format(1-a2, ".4f")}')

Ed: 0.4500
DTW(Learned w): 0.2500
DTW(w=100): 0.2500
